In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

import deeplabcut


Autosaving every 180 seconds


In [2]:
# READ X,Y and Likelihoods from a presaved CSV file
   
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]

    # load values
    data_array = np.array(data[3:])

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    for idx in range(1,len(labels)-1,3):
        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan

def load_csv_data(data_all):
    traces = np.array(data_all[0])
    #print (traces[0].shape)
    tracesx = []
    tracesy = []
    likelihoods = []

    for k in range(0,len(traces),1):
        tracesx.append(traces[k][:,0])
        tracesy.append(traces[k][:,1])
        likelihoods.append(traces[k][:,2])

    likelihoods = np.array(likelihoods)
    tracesx = np.array(tracesx)
    tracesy = np.array(tracesy)
    print ("tracex: ", tracesx.shape)
    print ("tracey: ", tracesy.shape)
    print ("likelihood: ", likelihoods.shape)
        
    return tracesx, tracesy, likelihoods

# MAKE VIDOES + LABELS 

# Robust outlier detection function to crop some of the DLC errors;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [3]:
# load a csv file; 

# OLD CSV DATA
if False:
    csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
    data_all = load_csv(csv_fname)
    tracesx, tracesy, likelihoods = load_csv_data(data_all)
# NEW INFERENCE FIXED DATA
else:
    # alterantive data:
    csv_fname= '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_inference_fixed.npz'
    traces = np.load(csv_fname)
    tracesx = traces['tracesx']
    tracesy = traces['tracesy']
    likelihoods = traces['probs']
    print ("tracesx: ", tracesx.shape)

    
#traces_all = np.stack((np.stack((tracesx,tracesy)),likelihoods))
print ("DONE Loading traces and likelihoods")

tracesx:  (56, 89989)
DONE Loading traces and likelihoods


In [53]:
fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/labels_array.npz'
data = np.load(fname, allow_pickle=True)
tracesx= data['tracesx']
tracesy= data['tracesy']
likelihoods= data['probs']
print (tracesx.shape)



(56, 89500)


In [3]:
fnames = [
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_54_23_358257_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02_54_39_170978_compressed/2020-3-16_02_54_39_170978_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02 57 41 995158_compressed/2020-3-16_02 57 41 995158_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03_57_56_902379_compressed/2020-3-16_03_57_56_902379_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03 54 54 231226_compressed/2020-3-16_03 54 54 231226_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_55_09_841582_compressed/2020-3-16_04_55_09_841582_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_58_11_998956_compressed/2020-3-16_04_58_11_998956_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 55 25 305681_compressed/2020-3-16_05 55 25 305681_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 58 27 193818_compressed/2020-3-16_05 58 27 193818_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 55 40 714236_compressed/2020-3-16_06 55 40 714236_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 58 43 678014_compressed/2020-3-16_06 58 43 678014_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_55_55_775234_compressed/2020-3-16_07_55_55_775234_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_59_00_362242_compressed/2020-3-16_07_59_00_362242_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_56_11_096689_compressed/2020-3-16_08_56_11_096689_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_59_17_534732_compressed/2020-3-16_08_59_17_534732_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09_59_34_731308_compressed/2020-3-16_09_59_34_731308_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09 56 26 362091_compressed/2020-3-16_09 56 26 362091_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10_59_50_448686_compressed/2020-3-16_10_59_50_448686_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10 56 41 406701_compressed/2020-3-16_10 56 41 406701_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_11_56_56_704655_compressed/2020-3-16_11_56_56_704655_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12 57 12 418305_compressed/2020-3-16_12 57 12 418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz'  
 
    
]

In [5]:
# Save median/single feature trackelts and likelihoods
# initialize start and ends of video to be analyzed
comments=False
for fname in fnames:
    print ('laoding:', fname)
    data = np.load(fname, allow_pickle=True)
    tracesx= data['tracesx']
    tracesy= data['tracesy']
    likelihoods= data['probs']
    #print (tracesx.shape)


    start = 0
    end = tracesx.shape[1]
    #end = 10000

    tracesx_mean =[]
    tracesy_mean =[]
    likelihoods_mean =[]
    threshold = 0.01

    total_nans = [0]*4

    comments = False
    for k in range(start,end,1):    
    #for k in [2940]:
        if k%5000==0:
            print (k, "/", end)

        tracesx_mean.append([])
        tracesy_mean.append([])

        if comments:
            print (k, " ##################################################")
        # PLOT MEAN FEATURE
        for p in range(0, tracesx.shape[0], 14):
            l = likelihoods[p:p+14,k]
            idx = np.where(l<threshold)[0]

            traces_localx = tracesx[p:p+14,k]
            traces_localx = np.delete(traces_localx, idx)

            # remove nans from data
            #idx_nan = np.where(np.isnan(traces_local)==True)[0]
            idx_nan = np.where(traces_localx==0)[0]
            traces_localx = np.delete(traces_localx, idx_nan)

            # reject outliers
            #traces_local = reject_outliers(traces_local, m = 4)

            x = np.mean(traces_localx,0)

            # search for y
            traces_localy = tracesy[p:p+14,k]
            traces_localy = np.delete(traces_localy, idx)

            # remove nans from data
            #idx_nan = np.where(np.isnan(traces_local)==True)[0]
            idx_nan = np.where(traces_localy==0)[0]
            traces_localy = np.delete(traces_localy, idx_nan)

            y = np.mean(traces_localy,0)

            if comments:
                print (p, "  XY: ", x,y)
            tracesx_mean[k].append(x)
            tracesy_mean[k].append(y)

            if np.isnan(x) or np.isnan(y):
                #print (x,y)
                total_nans[p//14]+=1
        if comments:
            print ("")
    tracesx_mean = np.array(tracesx_mean)
    tracesy_mean = np.array(tracesy_mean)

    print ("Total nans (missing frames) for each animal: ", total_nans)

    np.savez(fname[:-4]+'_single_feature_traces.npz', 
             tracesx=tracesx_mean,
             tracesy=tracesy_mean)

laoding: /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_54_23_358257_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
0 / 89988


/home/madeline/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/madeline/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


5000 / 89988
10000 / 89988
15000 / 89988
20000 / 89988
25000 / 89988
30000 / 89988
35000 / 89988
40000 / 89988
45000 / 89988
50000 / 89988
55000 / 89988
60000 / 89988
65000 / 89988
70000 / 89988
75000 / 89988
80000 / 89988
85000 / 89988
Total nans (missing frames) for each animal:  [11283, 15697, 17477, 16746]
laoding: /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
0 / 89988
5000 / 89988
10000 / 89988
15000 / 89988
20000 / 89988
25000 / 89988
30000 / 89988
35000 / 89988
40000 / 89988
45000 / 89988
50000 / 89988
55000 / 89988
60000 / 89988
65000 / 89988
70000 / 89988
75000 / 89988
80000 / 89988
85000 / 89988
Total nans (missing frames) for each animal:  [26792, 32776, 48951, 50062]
laoding: /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02_54_39_170978_compressed/2020-3-16_02_54_39_170978_co

10000 / 89989
15000 / 89989
20000 / 89989
25000 / 89989
30000 / 89989
35000 / 89989
40000 / 89989
45000 / 89989
50000 / 89989
55000 / 89989
60000 / 89989
65000 / 89989
70000 / 89989
75000 / 89989
80000 / 89989
85000 / 89989
Total nans (missing frames) for each animal:  [11774, 13430, 14363, 18930]
laoding: /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09_59_34_731308_compressed/2020-3-16_09_59_34_731308_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
0 / 89989
5000 / 89989
10000 / 89989
15000 / 89989
20000 / 89989
25000 / 89989
30000 / 89989
35000 / 89989
40000 / 89989
45000 / 89989
50000 / 89989
55000 / 89989
60000 / 89989
65000 / 89989
70000 / 89989
75000 / 89989
80000 / 89989
85000 / 89989
Total nans (missing frames) for each animal:  [9373, 9016, 4445, 16945]
laoding: /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09 56 26 362091_compressed/2020-3-16_09 56 26 362091_compressedDLC_resn

In [38]:
print (tracesx_mean.shape)
plt.plot(tracesx_mean[:,1])
plt.show()

(10000, 4)


Total nans (missing frames) for each animal:  [31993, 44883, 64551, 77055]
DONE Loading mean traces


In [27]:
print ("DONE Loading mean traces")
np.savez('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_single_feature_traces_WITH_INFERENCE.npz', 
        tracesx=tracesx_mean,
        tracesy=tracesy_mean)

DONE Loading mean traces


In [39]:
print (tracesx_mean)

[[284.815     nan     nan     nan]
 [285.203     nan     nan     nan]
 [285.049     nan     nan     nan]
 ...
 [    nan     nan     nan     nan]
 [    nan     nan     nan     nan]
 [    nan     nan     nan     nan]]


In [1]:
# filter data
total=[1964, 4534, 5890, 9721]

In [4]:
import numpy as np
print (np.array(total)/16572)

[0.11851315 0.27359401 0.35541878 0.58659184]
